In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.font_manager
from matplotlib import rcParams, cm
import scipy.stats as ss
import h5py
import glob
import seaborn as sns
import time
import os
import itertools
import timeit
import networkx as nx
import scipy.io

#import plotly.graph_objects as go 

#For ROI Maps
from scipy.io import loadmat
from skimage import measure
from scipy import ndimage

#For Fitting Baselines
from scipy.optimize import curve_fit



In [2]:
def makeFilenames(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/{}_10Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_trace_final.h5'
    return directories, filenames

In [3]:
def makeFilenames_2(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/10Hz/{}_10Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_trace_final.h5'
    return directories, filenames

In [4]:
def combineFilenames(base, dirs, fnDict):
    #Combine Filenames into list for multiple directories with a common base.  fnDict is a dictionary of the filename for each directory
    outFiles = []
    for d in dirs:
        outFiles.append(os.path.join(base+d+fnDict[d]))
    return outFiles

In [5]:
#Initialize Input Values
#
Base = '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/'
Mice, Days = ([608448,608451,608452], ['01062020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files1 = combineFilenames(Base, Folders, Filenames)
#Combine All Filenames into one List
Mice, Days = ([608450,611311,602101], ['03092020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files2 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608448], ['01172020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files3 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608451,608452,611111,602101,612535], ['07082020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files4 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([615883], ['02052021','03122021'])
Folders, Filenames = makeFilenames_2(Mice, Days)
Files5 = combineFilenames(Base, Folders, Filenames)
Files= Files1+Files2+Files3+Files4+Files5
Files

['/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608448/01062020/608448_10Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608451/01062020/608451_10Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608452/01062020/608452_10Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608450/03092020/608450_10Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/611311/03092020/611311_10Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/602101/03092020/602101_10Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608448/01172020/6084

In [6]:
def loadFiles(allFiles, fieldName,multiIndex):
    firstf = Files[0]
    firstdir = firstf.split('/')[6]+'_'+firstf.split('/')[7]
    f = h5py.File(firstf, 'r')
    if multiIndex: #Adjust indexing for first DataFrame if using a multi-index
        first_array = np.array(f[fieldName]).T
        index_iterables = [[firstdir], range(first_array.shape[1])]
        index = pd.MultiIndex.from_product(index_iterables, names=['mouse','cell_num'])
        df = pd.DataFrame(data=first_array, columns=index)
    else:
        df = pd.DataFrame(data=np.array(f[fieldName]).T, columns=[firstdir])
    df.index.name='Time'    

    df_size = df.shape[0] #Determine Size of DataFrame
    for f in Files[1:]: #Loop through remaining files and add to DataFrame
        fdir = f.split('/')[6]+'_'+f.split('/')[7] #Current Filename/Dir
        f = h5py.File(f, 'r')
        if multiIndex: #Iteratively add multiIndexed DataFrames
            data_arr = np.array(f[fieldName]).T
            index_iterables = [[fdir], range(data_arr.shape[1])]
            index = pd.MultiIndex.from_product(index_iterables, names=['mouse', 'cell_num'])
            df = df.join(pd.DataFrame(data=data_arr, columns=index))
    drop_index = df.index[df.isna().any(axis='columns')]
    df = df.drop(drop_index)
    return df

In [7]:
processed_traces_df=loadFiles(Files, 'trace',True)

In [8]:
onset_binary_traces_df=loadFiles(Files, 'onset_binary_trace',True)
onset_binary_traces_df

mouse    608448_01062020                                               ...  \
cell_num              0    1    2    3    4    5    6    7    8    9   ...   
Time                                                                   ...   
0                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
1                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
2                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
3                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
4                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
...                  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
43195                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43196                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43197                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43198                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43199                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

mouse    615883_03122021                                               
cell_num              42   43   44   45   46   47   48   49   50   51  
Time                                                                   
0                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...                  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
43195                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43196                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43197                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43198                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43199                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[43200 rows x 3324 columns]

In [9]:
binary_traces_df=loadFiles(Files, 'binary_trace',True)

In [10]:
def makeFilenames(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/{}_10Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'LFP_ts.h5'
    return directories, filenames

In [11]:
def makeFilenames_2(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/10Hz/{}_10Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'LFP_ts.h5'
    return directories, filenames

In [12]:
def combineFilenames(base, dirs, fnDict):
    #Combine Filenames into list for multiple directories with a common base.  fnDict is a dictionary of the filename for each directory
    outFiles = []
    for d in dirs:
        outFiles.append(os.path.join(base+d+fnDict[d]))
    return outFiles

In [13]:
#Initialize Input Values
#
Base = '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/'
Mice, Days = ([608451,608452], ['01062020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files1 = combineFilenames(Base, Folders, Filenames)
#Combine All Filenames into one List
Mice, Days = ([608450,611311,602101], ['03092020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files2 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608448], ['01172020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files3 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608451,608452,611111,602101,612535], ['07082020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files4 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([615883], ['02052021','03122021'])
Folders, Filenames = makeFilenames_2(Mice, Days)
Files5 = combineFilenames(Base, Folders, Filenames)
LFP_Files= Files1+Files2+Files3+Files4+Files5


In [14]:
# Find a good way to store stim onsets and offsets 
def load_onset_offset_2(files,fieldName):
    firstf = Files[0]
    
    firstdir = firstf.split('/')[6]+'_'+firstf.split('/')[7]
    f = h5py.File(firstf, 'r')
    data=np.array(f[fieldName])-1
    data=np.reshape(data,(1,5))
    df = pd.DataFrame(data)
    df.index=[firstdir]
   
    for f in files[0:]: #Loop through remaining files and add to DataFrame
        fdir = f.split('/')[6]+'_'+f.split('/')[7] #Current Filename/Dir
        f = h5py.File(f, 'r')
        data=np.array(f[fieldName])-1
        data=np.reshape(data,(1,5))
        df1 = pd.DataFrame(data)
        df1.index=[fdir]
        df = pd.concat([df,df1])
    df=df.astype(int)
    return df

In [15]:
stim_onsets_df=load_onset_offset_2(LFP_Files,'Stim_onset')
stim_offsets_df=load_onset_offset_2(LFP_Files,'Stim_offset')
stim_time=stim_offsets_df-stim_onsets_df+1
Total_frames_stim = pd.DataFrame(stim_time.sum(axis=1))

In [16]:
# Exclude the min after stim
def get_baseline_frames_index(sess_no):
    stim_on_inds=stim_onsets_df.loc[sess_no,:]
    stim_off_inds=stim_offsets_df.loc[sess_no,:]

    baseline_indices=np.arange(0,stim_on_inds[0])
    for i in np.arange(0,4,1):
        b=np.arange(stim_off_inds[i]+1200,stim_on_inds[i+1])
        baseline_indices=np.concatenate((baseline_indices,b))
    b= np.arange(stim_off_inds[4]+1200,42000,1)
    baseline_indices=np.concatenate((baseline_indices,b))
    return baseline_indices

In [17]:
def get_stim_frames_index(sess_no):
    stim_on_inds=stim_onsets_df.loc[sess_no,:]
    stim_off_inds=stim_offsets_df.loc[sess_no,:]
    stim_indices=np.arange(stim_on_inds[0],stim_off_inds[0]+1,1)
    for i in np.arange(1,5,1):
        b=np.arange(stim_on_inds[i],stim_off_inds[i]+1,1)
        stim_indices=np.concatenate((stim_indices,b))
    return stim_indices

In [18]:
def remove_nan_motion_frames(sess_name):
    motion=np.array(motion_df.loc[sess_name,:])
    motion=np.squeeze(motion)
    nan_motion_frames=np.argwhere(np.isnan(motion))
    baseline_indices=get_baseline_frames_index(sess_name)
    c=np.intersect1d(baseline_indices,nan_motion_frames)
    common_indices=np.setdiff1d(baseline_indices,c)
    return common_indices

In [19]:
def makeFilenames_motion(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/{}_10Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_motion.h5'
    return directories, filenames
def makeFilenames_motion_2(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/10Hz/{}_10Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_motion.h5'
    return directories, filenames
Base = '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/'
Mice, Days = ([608448,608451,608452], ['01062020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files1 = combineFilenames(Base, Folders, Filenames)
#Combine All Filenames into one List
Mice, Days = ([608450,611311,602101], ['03092020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files2 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608448], ['01172020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files3 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608451,608452,611111,602101,612535], ['07082020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files4 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([615883], ['02052021','03122021'])
Folders, Filenames = makeFilenames_motion_2(Mice, Days)
Files5 = combineFilenames(Base, Folders, Filenames)
motion_Files= Files1+Files2+Files3+Files4+Files5

In [20]:
# Find a good way to store stim onsets and offsets 
def load_motion(Files,fieldName):
    firstf = Files[0]
    firstdir = firstf.split('/')[6]+'_'+firstf.split('/')[7]
    f = h5py.File(firstf, 'r')
    data=np.array(f[fieldName])
    df = pd.DataFrame(data)
    df.index=[firstdir]
   
    for f in Files[1:]: #Loop through remaining files and add to DataFrame
        fdir = f.split('/')[6]+'_'+f.split('/')[7] #Current Filename/Dir
        f = h5py.File(f, 'r')
        data=np.array(f[fieldName])
        df1 = pd.DataFrame(data)
        df1.index=[fdir]
        df = pd.concat([df,df1])
        #drop last minute
        df = df.iloc[: , :-1200]
    return df

In [21]:
motion_df=load_motion(motion_Files,'speed_trace')
moving_period_df=load_motion(motion_Files,'moving_period')
stationary_period_df=load_motion(motion_Files,'stationary_period')
acceleration_period_df=load_motion(motion_Files,'aceleration_period')
motion_onsets_df=load_motion(motion_Files,'motion_onset_with')
motion_onsets_df_without=load_motion(motion_Files,'motion_onset_without')

In [22]:
def baseline_firing_rate_cell_wise(traces_df,sess_no):
    onset_binary_traces=traces_df[sess_no]
    onset_binary_traces=onset_binary_traces.T
    baseline_indices=get_baseline_frames_index(sess_no)
    temp_df=onset_binary_traces.loc[:,baseline_indices]
    Ncells = temp_df.shape[0]
    out= np.zeros((Ncells,1))
    for cell in np.arange(0,Ncells,1):
        #out[cell]=out[cell]+np.array(np.where(temp_df.loc[cell,:]==1)).size
        out[cell]=out[cell]+np.sum(temp_df.loc[cell,:])
    tot_frames=(baseline_indices.shape[0])
    p=out/tot_frames*1200
    
    return p,np.mean(p)

In [25]:
# Method 1
sess_list=[]
for mouse, cell_df in onset_binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
sess_list
mean_p=np.zeros(len(sess_list))

for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    mean_p[i]=(baseline_firing_rate_cell_wise(onset_binary_traces_df,sess_name)[1])
mean_p

array([1.67471029, 3.08533303, 2.17592451, 2.09689569, 1.98206658,
       2.26821755, 3.0444662 , 2.87247893, 3.6658552 , 2.84028403,
       3.0323705 , 3.93494979, 2.41478742, 2.53341471])

In [26]:
## Make a dataframe with sess_list and mean_p
df = {'Sess list' : pd.Series(sess_list), 'Mean baseline firing rate' : pd.Series(mean_p)}
mean_p_df=pd.DataFrame(df)
mean_p_df

,Sess list,Mean baseline firing rate
0,602101_03092020,1.674710
1,602101_07082020,3.085333
2,608448_01062020,2.175925
3,608448_01172020,2.096896
4,608450_03092020,1.982067
5,608451_01062020,2.268218
6,608451_07082020,3.044466
7,608452_01062020,2.872479
8,608452_07082020,3.665855
9,611111_07082020,2.840284


In [27]:
## Split them as GCamp7 and somaGCamp7 - think of a clever way!
#GCamp7-608448,608450,608452,612535,615883
#somaGCamp7-6085451,602101,611111,611311
if sess_list[0].find('602101') != -1: 
    print ("Contains given substring ") 

Contains given substring 


In [28]:
G_baseline_firing_rate_10Hz=[mean_p[2],mean_p[3],mean_p[4],mean_p[7],mean_p[8],mean_p[11]]
soma_baseline_firing_rate_10Hz =[mean_p[0],mean_p[1],mean_p[5],mean_p[6],mean_p[9],mean_p[10],mean_p[12]]
print(np.mean(G_baseline_firing_rate_10Hz),np.mean(soma_baseline_firing_rate_10Hz))
print(np.std(G_baseline_firing_rate_10Hz),np.std(soma_baseline_firing_rate_10Hz))
print(np.median(G_baseline_firing_rate_10Hz),np.median(soma_baseline_firing_rate_10Hz))
print(ss.ttest_ind(G_baseline_firing_rate_10Hz,soma_baseline_firing_rate_10Hz))

2.7880284507906787 2.622881289646681
0.7740640064182476 0.48911851880563595
2.5242017175093556 2.8402840284028406
Ttest_indResult(statistic=0.42887142186687943, pvalue=0.6762951169532849)


In [29]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Event_rate_10Hz_per_min_final.h5", 'w')
label=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    mean_p=(baseline_firing_rate_cell_wise(onset_binary_traces_df,sess_name)[1])
    mean_p_bin=(baseline_firing_rate_cell_wise(binary_traces_df,sess_name)[1])
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i]+'_onset', data=mean_p)
    dset2 = f1.create_dataset(sess_list[i]+'_bin', data=mean_p_bin)
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03092020', '602101_07082020', '608448_01062020', '608448_01172020', '608450_03092020', '608451_01062020', '608451_07082020', '608452_01062020', '608452_07082020', '611111_07082020', '611311_03092020', '612535_07082020', '615883_02052021', '615883_03122021']


In [23]:
def upper_tri_masking(A):
    m = A.shape[0]
    r = np.arange(m)
    mask = r[:,None] < r
    return A[mask]

In [24]:
# Redo this function!
def baseline_corr(sess_name):
    traces= binary_traces_df[sess_name].T
    speed_df=np.array(traces)
    corr_matrix=np.corrcoef(speed_df)
    corr_list=upper_tri_masking(corr_matrix)
    mean_corr=np.nanmean(corr_list)
    median_corr=np.nanmedian(corr_list)
    return mean_corr,median_corr

In [25]:
def corr_before_first_stim(sess_name):
    traces= binary_traces_df[sess_name].T
    Ncells=traces.shape[0]
    common_indices=remove_nan_motion_frames(sess_name)
    stim_on_inds=stim_onsets_df.loc[sess_name,:]
    indices=np.arange(common_indices[0],stim_on_inds[0]+1,1)
    speed_index = indices
    speed_df=np.array(traces.loc[:,indices])
    corr_matrix=np.corrcoef(speed_df)
    obs_corr_list=upper_tri_masking(corr_matrix)
    mean_corr=np.nanmean(obs_corr_list)
    median_corr=np.nanmedian(obs_corr_list)
    return mean_corr,median_corr

In [26]:
def corr_before_first_stim2(sess_name):
    traces= binary_traces_df[sess_name].T
    Ncells=traces.shape[0]
    common_indices=remove_nan_motion_frames(sess_name)
    stim_on_inds=stim_onsets_df.loc[sess_name,:]
    indices=np.arange(common_indices[0],stim_on_inds[0]+1,1)
    speed_index = indices
    speed_df=np.array(traces.loc[:,indices])
    corr_matrix=np.corrcoef(speed_df)
    obs_corr_list=upper_tri_masking(corr_matrix)
    mean_corr=np.nanmean(obs_corr_list)
    median_corr=np.nanmedian(obs_corr_list)
    return obs_corr_list

In [34]:
sess_list=[]
for mouse, cell_df in onset_binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
sess_list
mean_corr=np.zeros(len(sess_list))

for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    mean_corr[i],_=baseline_corr(sess_name)
mean_corr

/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


array([0.02423728, 0.02387254, 0.00813154, 0.01278351, 0.0122976 ,
       0.01264545, 0.01743308, 0.00880574, 0.00985052, 0.01845124,
       0.00814773, 0.01333127, 0.00891723, 0.00984987])

In [28]:
sess_list=[]
for mouse, cell_df in onset_binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
sess_list
median_corr=np.zeros(len(sess_list))

for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    _,median_corr[i]=corr_before_first_stim(sess_name)
median_corr

array([-0.00361897, -0.00124769, -0.00783264, -0.00609209, -0.00411894,
       -0.00505094, -0.00020826, -0.00446961, -0.00561763, -0.00339681,
       -0.00685324, -0.00515387, -0.00427042, -0.00919991])

In [29]:
## Make a dataframe with sess_list and mean_p
df = {'Sess list' : pd.Series(sess_list), 'Median baseline corr' : pd.Series(median_corr)}
median_corr_df=pd.DataFrame(df)
median_corr_df

,Sess list,Median baseline corr
0,602101_03092020,-0.003619
1,602101_07082020,-0.001248
2,608448_01062020,-0.007833
3,608448_01172020,-0.006092
4,608450_03092020,-0.004119
5,608451_01062020,-0.005051
6,608451_07082020,-0.000208
7,608452_01062020,-0.004470
8,608452_07082020,-0.005618
9,611111_07082020,-0.003397


In [37]:
G_baseline_firing_corr_10Hz=[mean_corr[2],mean_corr[3],mean_corr[4],mean_corr[7],mean_corr[8],mean_corr[11]]
soma_baseline_firing_corr_10Hz =[mean_corr[0],mean_corr[1],mean_corr[5],mean_corr[6],mean_corr[9],mean_corr[10],mean_corr[12]]
print(np.mean(G_baseline_firing_corr_10Hz),np.mean(soma_baseline_firing_corr_10Hz))
print(np.std(G_baseline_firing_corr_10Hz),np.std(soma_baseline_firing_corr_10Hz))
print(np.median(G_baseline_firing_corr_10Hz),np.median(soma_baseline_firing_corr_10Hz))
print(ss.ttest_ind(G_baseline_firing_corr_10Hz,soma_baseline_firing_corr_10Hz))
print(ss.ranksums(G_baseline_firing_corr_10Hz,soma_baseline_firing_corr_10Hz))

0.010299730383348573 0.015030985002482594
0.0014640556855052853 0.004997382925627378
0.009875410949621431 0.015561313359549947
Ttest_indResult(statistic=-2.0588248606070576, pvalue=0.06399492488485481)
RanksumsResult(statistic=-1.8571428571428572, pvalue=0.06329083223334525)


In [38]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_mean_corr_10Hz_final.h5", 'w')
label=np.zeros(len(sess_list))
mean_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    mean_corr[i],_=baseline_corr(sess_name)
    
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i], data=mean_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03092020', '602101_07082020', '608448_01062020', '608448_01172020', '608450_03092020', '608451_01062020', '608451_07082020', '608452_01062020', '608452_07082020', '611111_07082020', '611311_03092020', '612535_07082020', '615883_02052021', '615883_03122021']


/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [30]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_median_corr_10Hz_before_first_stim_final.h5", 'w')
label=np.zeros(len(sess_list))
median_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    _,median_corr[i]=corr_before_first_stim(sess_name)
    
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i], data=median_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03092020', '602101_07082020', '608448_01062020', '608448_01172020', '608450_03092020', '608451_01062020', '608451_07082020', '608452_01062020', '608452_07082020', '611111_07082020', '611311_03092020', '612535_07082020', '615883_02052021', '615883_03122021']


/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [40]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_median_corr_10Hz_final.h5", 'w')
label=np.zeros(len(sess_list))
median_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    _,median_corr[i]=baseline_corr(sess_name)
    
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i], data=median_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03092020', '602101_07082020', '608448_01062020', '608448_01172020', '608450_03092020', '608451_01062020', '608451_07082020', '608452_01062020', '608452_07082020', '611111_07082020', '611311_03092020', '612535_07082020', '615883_02052021', '615883_03122021']


In [42]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_median_corr_10Hz_before_first_stim_final.h5", 'w')
label=np.zeros(len(sess_list))
median_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    _,median_corr[i]=corr_before_first_stim(sess_name)
    
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i], data=median_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03092020', '602101_07082020', '608448_01062020', '608448_01172020', '608450_03092020', '608451_01062020', '608451_07082020', '608452_01062020', '608452_07082020', '611111_07082020', '611311_03092020', '612535_07082020', '615883_02052021', '615883_03122021']


In [43]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_corr_list_before_first_stim_10Hz.h5", 'w')
label=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    corr_list=corr_before_first_stim2(sess_name)
    
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i], data=corr_list)
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03092020', '602101_07082020', '608448_01062020', '608448_01172020', '608450_03092020', '608451_01062020', '608451_07082020', '608452_01062020', '608452_07082020', '611111_07082020', '611311_03092020', '612535_07082020', '615883_02052021', '615883_03122021']


## 145 Hz sessions 

In [44]:
def makeFilenames(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/{}_145Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_trace_final.h5'
    return directories, filenames

In [45]:
def makeFilenames_2(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/145Hz/{}_145Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_trace_final.h5'
    return directories, filenames

In [46]:
def combineFilenames(base, dirs, fnDict):
    #Combine Filenames into list for multiple directories with a common base.  fnDict is a dictionary of the filename for each directory
    outFiles = []
    for d in dirs:
        outFiles.append(os.path.join(base+d+fnDict[d]))
    return outFiles

In [47]:
#Initialize Input Values
#
Base = '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/'
Mice, Days = ([608448,608451,608452], ['01102020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files1 = combineFilenames(Base, Folders, Filenames)
#Combine All Filenames into one List
Mice, Days = ([611111,611311,602101], ['03112020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files2 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608450], ['01172020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files3 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608451,608452,611111], ['07012020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files4 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([612535,602101], ['07022020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files5 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([615883], ['02052021','03122021'])
Folders, Filenames = makeFilenames_2(Mice, Days)
Files6 = combineFilenames(Base, Folders, Filenames)
Files= Files1+Files2+Files3+Files4+Files5+Files6
Files

['/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608448/01102020/608448_145Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608451/01102020/608451_145Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608452/01102020/608452_145Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/611111/03112020/611111_145Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/611311/03112020/611311_145Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/602101/03112020/602101_145Hz_AudioVisual/motion_corrected/processed_trace_final.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608450/0117202

In [48]:
processed_traces_df=loadFiles(Files, 'trace',True)

In [49]:
onset_binary_traces_df=loadFiles(Files, 'onset_binary_trace',True)
onset_binary_traces_df

mouse    608448_01102020                                               ...  \
cell_num              0    1    2    3    4    5    6    7    8    9   ...   
Time                                                                   ...   
0                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
1                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
2                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
3                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
4                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
...                  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
43195                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43196                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43197                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43198                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
43199                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

mouse    615883_03122021                                               
cell_num              74   75   76   77   78   79   80   81   82   83  
Time                                                                   
0                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4                    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...                  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
43195                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43196                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43197                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43198                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
43199                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[43200 rows x 3371 columns]

In [50]:
binary_traces_df=loadFiles(Files, 'binary_trace',True)

In [51]:
def makeFilenames(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/{}_145Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'LFP_ts.h5'
    return directories, filenames

In [52]:
def makeFilenames_2(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/145Hz/{}_145Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'LFP_ts.h5'
    return directories, filenames

In [53]:
def combineFilenames(base, dirs, fnDict):
    #Combine Filenames into list for multiple directories with a common base.  fnDict is a dictionary of the filename for each directory
    outFiles = []
    for d in dirs:
        outFiles.append(os.path.join(base+d+fnDict[d]))
    return outFiles

In [54]:
#Initialize Input Values
#
Base = '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/'
Mice, Days = ([608448,608451,608452], ['01102020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files1 = combineFilenames(Base, Folders, Filenames)
#Combine All Filenames into one List
Mice, Days = ([611111,611311,602101], ['03112020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files2 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608450], ['01172020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files3 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608451,608452,611111], ['07012020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files4 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([612535,602101], ['07022020'])
Folders, Filenames = makeFilenames(Mice, Days)
Files5 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([615883], ['02052021','03122021'])
Folders, Filenames = makeFilenames_2(Mice, Days)
Files6 = combineFilenames(Base, Folders, Filenames)
LFP_Files= Files1+Files2+Files3+Files4+Files5+Files6
LFP_Files

['/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608448/01102020/608448_145Hz_AudioVisual/motion_corrected/LFP_ts.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608451/01102020/608451_145Hz_AudioVisual/motion_corrected/LFP_ts.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608452/01102020/608452_145Hz_AudioVisual/motion_corrected/LFP_ts.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/611111/03112020/611111_145Hz_AudioVisual/motion_corrected/LFP_ts.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/611311/03112020/611311_145Hz_AudioVisual/motion_corrected/LFP_ts.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/602101/03112020/602101_145Hz_AudioVisual/motion_corrected/LFP_ts.h5',
 '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/608450/01172020/608450_145Hz_AudioVisual/motion_corrected/LFP_ts.h5',
 '/home/hanlabadmins/eng_handata/e

In [55]:
# Find a good way to store stim onsets and offsets 
def load_onset_offset_2(files,fieldName):
    firstf = Files[0]
    
    firstdir = firstf.split('/')[6]+'_'+firstf.split('/')[7]
    f = h5py.File(firstf, 'r')
    data=np.array(f[fieldName])-1
    data=np.reshape(data,(1,5))
    df = pd.DataFrame(data)
    df.index=[firstdir]
   
    for f in files[1:]: #Loop through remaining files and add to DataFrame
        fdir = f.split('/')[6]+'_'+f.split('/')[7] #Current Filename/Dir
        f = h5py.File(f, 'r')
        data=np.array(f[fieldName])-1
        data=np.reshape(data,(1,5))
        df1 = pd.DataFrame(data)
        df1.index=[fdir]
        df = pd.concat([df,df1])
    df=df.astype(int)
    return df

In [56]:
stim_onsets_df=load_onset_offset_2(LFP_Files,'Stim_onset')
stim_offsets_df=load_onset_offset_2(LFP_Files,'Stim_offset')
stim_time=stim_offsets_df-stim_onsets_df+1
Total_frames_stim = pd.DataFrame(stim_time.sum(axis=1))

In [57]:
def makeFilenames_motion(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/{}_145Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_motion.h5'
    return directories, filenames

def makeFilenames_motion_2(mouse_numbers, mouse_days):
    #Input lists of mouse numbers, days, and sessions to permute to give out directories and dictionary for filename
    directories = []
    filenames = {}
    for m_num in mouse_numbers:
        for m_day in mouse_days:
            mouse_dir = '{}/{}/145Hz/{}_145Hz_AudioVisual/motion_corrected/'.format(m_num, m_day,m_num)
            directories.append(mouse_dir)
                #filenames[mouse_dir] = 'trace_kyleFinal_BinaryVideo.hdf5'
            filenames[mouse_dir] = 'processed_motion.h5'
    return directories, filenames

#Initialize Input Values
#
Base = '/home/hanlabadmins/eng_handata/eng_research_handata2/Sudi_Sridhar/'
Mice, Days = ([608448,608451,608452], ['01102020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files1 = combineFilenames(Base, Folders, Filenames)
#Combine All Filenames into one List
Mice, Days = ([611111,611311,602101], ['03112020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files2 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608450], ['01172020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files3 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([608451,608452,611111], ['07012020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files4 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([612535,602101], ['07022020'])
Folders, Filenames = makeFilenames_motion(Mice, Days)
Files5 = combineFilenames(Base, Folders, Filenames)
Mice, Days = ([615883], ['02052021','03122021'])
Folders, Filenames = makeFilenames_motion_2(Mice, Days)
Files6 = combineFilenames(Base, Folders, Filenames)
motion_Files= Files1+Files2+Files3+Files4+Files5+Files6
motion_Files

motion_df=load_motion(motion_Files,'speed_trace')

In [58]:
# Method 1
sess_list=[]
for mouse, cell_df in onset_binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)

In [59]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Event_rate_145Hz_per_min_final.h5", 'w')
label=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    mean_p=(baseline_firing_rate_cell_wise(onset_binary_traces_df,sess_name)[1])
    mean_p_bin=(baseline_firing_rate_cell_wise(binary_traces_df,sess_name)[1])
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i]+'_onset', data=mean_p)
    dset2 = f1.create_dataset(sess_list[i]+'_bin', data=mean_p_bin)
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03112020', '602101_07022020', '608448_01102020', '608450_01172020', '608451_01102020', '608451_07012020', '608452_01102020', '608452_07012020', '611111_03112020', '611111_07012020', '611311_03112020', '612535_07022020', '615883_02052021', '615883_03122021']


In [60]:
#608448,608450,608452,612535,615883(check) GCamp7
if sess_list[4].find('608448') != -1 or sess_list[4].find('608450') != -1 or sess_list[4].find('608452') != -1 or sess_list[4].find('612535') != -1 or sess_list[4].find('615883') != -1: 
    print ("Contains given substring ") 

In [61]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_mean_corr_145Hz_final.h5", 'w')
label=np.zeros(len(sess_list))
mean_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    mean_corr[i],_=baseline_corr(sess_name)
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    
    dset1 = f1.create_dataset(sess_list[i], data=mean_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03112020', '602101_07022020', '608448_01102020', '608450_01172020', '608451_01102020', '608451_07012020', '608452_01102020', '608452_07012020', '611111_03112020', '611111_07012020', '611311_03112020', '612535_07022020', '615883_02052021', '615883_03122021']


/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [62]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_mean_corr_145Hz_before_first_stim_final.h5", 'w')
label=np.zeros(len(sess_list))
mean_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    mean_corr[i],_=corr_before_first_stim(sess_name)
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    
    dset1 = f1.create_dataset(sess_list[i], data=mean_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03112020', '602101_07022020', '608448_01102020', '608450_01172020', '608451_01102020', '608451_07012020', '608452_01102020', '608452_07012020', '611111_03112020', '611111_07012020', '611311_03112020', '612535_07022020', '615883_02052021', '615883_03122021']


In [63]:
## Make a dataframe with sess_list and mean_p
df = {'Sess list' : pd.Series(sess_list), 'Mean baseline corr' : pd.Series(mean_corr)}
mean_corr_df=pd.DataFrame(df)
mean_corr_df

,Sess list,Mean baseline corr
0,602101_03112020,0.008641
1,602101_07022020,0.024874
2,608448_01102020,0.008463
3,608450_01172020,0.010640
4,608451_01102020,0.016344
5,608451_07012020,0.008271
6,608452_01102020,0.013337
7,608452_07012020,0.012879
8,611111_03112020,0.048894
9,611111_07012020,0.011201


In [64]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_median_corr_145Hz_final.h5", 'w')

label=np.zeros(len(sess_list))
median_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    _,median_corr[i]=baseline_corr(sess_name)
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    
    dset1 = f1.create_dataset(sess_list[i], data=median_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03112020', '602101_07022020', '608448_01102020', '608450_01172020', '608451_01102020', '608451_07012020', '608452_01102020', '608452_07012020', '611111_03112020', '611111_07012020', '611311_03112020', '612535_07022020', '615883_02052021', '615883_03122021']


/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hanlabadmins/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [65]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_median_corr_145Hz_before_first_stim_final.h5", 'w')

label=np.zeros(len(sess_list))
median_corr=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    _,median_corr[i]=corr_before_first_stim(sess_name)
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    
    dset1 = f1.create_dataset(sess_list[i], data=median_corr[i])
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03112020', '602101_07022020', '608448_01102020', '608450_01172020', '608451_01102020', '608451_07012020', '608452_01102020', '608452_07012020', '611111_03112020', '611111_07012020', '611311_03112020', '612535_07022020', '615883_02052021', '615883_03122021']


In [66]:
## Store in .h5 files 
sess_list=[]
for mouse, cell_df in binary_traces_df.groupby(level='mouse', axis=1):
    sess_list.append(mouse)
print(sess_list)
f1 = h5py.File("Baseline_corr_list_before_first_stim_145Hz_final.h5", 'w')
label=np.zeros(len(sess_list))
for i in np.arange(0,len(sess_list),1):
    sess_name=sess_list[i]
    corr_list=corr_before_first_stim2(sess_name)
    
    if sess_list[i].find('608448') != -1 or sess_list[i].find('608450') != -1 or sess_list[i].find('608452') != -1 or sess_list[i].find('612535') != -1 or sess_list[i].find('615883') != -1: 
        label[i]=1
    if sess_list[i].find('602101') != -1 or sess_list[i].find('608451') != -1 or sess_list[i].find('611311') != -1 or sess_list[i].find('611111') != -1: 
        label[i]=2
    dset1 = f1.create_dataset(sess_list[i], data=corr_list)
dset3=f1.create_dataset('GCamp_version', data=label)
f1.close()

['602101_03112020', '602101_07022020', '608448_01102020', '608450_01172020', '608451_01102020', '608451_07012020', '608452_01102020', '608452_07012020', '611111_03112020', '611111_07012020', '611311_03112020', '612535_07022020', '615883_02052021', '615883_03122021']
